In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate
sym.init_session(use_latex=True)

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



Derivada central

In [2]:
def DerivadaCentral(f,x,h):
    
    d = 0.
    
    if h != 0:
        d = (f(x+h) - f(x-h))/(2*h)
        
    return d

Newton-Raphson

In [3]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn
    
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

Integración simple

In [4]:
class Simpson_tres_octavos_Simple:

    def __init__(self,a,b,f):
        self.a = a
        self.b = b
        self.f = f
        self.h = (b-a)/3
        self.x = np.linspace(a,b,100)
        self.h_ = self.x[1] - self.x[0]

    def GetIntegral(self):

        self.Integral = (self.f(self.a) + 3*self.f((2*self.a + self.b)/3) + self.f((self.a + 2*self.b)/3) + self.f(self.b))*3*self.h/8

        return self.Integral

Integración Compuesta

In [5]:
class Integrator:
    
    def __init__(self,x,f):
        
        self.x = x
        self.h = self.x[1] - self.x[0]
        self.y = f(self.x)
        self.f = f
        
        self.Integral = 0.

# Integración del método del trapecio y Simpson 1/3 en general

class trapecio_compuesto(Integrator):
    def __init__(self,x,f):
        Integrator.__init__(self,x,f)

    def GetIntegral(self):
        
        self.Integral += 0.5*(self.y[0]+self.y[-1])
        
        #self.Integral += np.sum( self.y[1:-1] )
        
        for i in range(1,self.y.shape[0]-1):
            self.Integral += self.y[i]
        
        self.Integral *= self.h
        
        return self.Integral

class Simpson_un_tercio_compuesto(Integrator):
    
    def __init__(self,x,f):
        Integrator.__init__(self,x,f)
        
    def GetIntegral(self):
        
        self.Integral = 0.
        
        self.Integral += self.y[0] + self.y[-1]
        
        for i in range( len(self.y[1:-2]) ):
            
            if i%2 == 0:
                self.Integral += 4*self.y[i+1]
            else:
                self.Integral += 2*self.y[i+1]
          
        return self.Integral*self.h/3
    
class Simpson_tres_octavos_compuesto(Integrator):
    
    def __init__(self,x,f):
        Integrator.__init__(self,x,f)
        
    def GetIntegral(self):
        
        self.Integral = 0.
        self.Integral += self.y[0] + self.y[-1]

        for i in range(len(self.x[1:-1])):
            
            if i%3 == 0:
                self.Integral += 2*self.y[i+1]
            
            else:
                self.Integral += 3*self.y[i+1]
          
        return self.Integral*self.h*3/8
    
    def GetDerivative(self):
        
        d = self.f(self.x + 2*self.h) - 4*self.f(self.x + self.h) + 6*self.f(self.x) - 4*self.f(self.x - self.h) + self.f(self.x - 2*self.h)
        d /= self.h**4
        
        
        return d
    
    def GetError(self):
        
        d = self.GetDerivative()
        max_ = np.max(np.abs(d))
        
        self.error = (self.x[-1]-self.x[0])*self.h**4*max_/180
        
        return self.error

Gauss-Legendre

In [6]:
#Según magistral

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)
n = 6

def GetLegendre(n,x,y):
    
    y = (x**2 - 1)**n
    
    poly = sym.diff( y,x,n )/(2**n*np.math.factorial(n))
    
    return poly

Legendre = []
DLegendre = []

for i in range(n+1):
    Poly = GetLegendre(i,x,y)
    Legendre.append(Poly)
    DLegendre.append(sym.diff(Poly,x,1))

Roots, Weights = np.polynomial.legendre.leggauss(n)

a = 0.
b = 0.25*np.pi
f = lambda x: np.sin(x)
exact = -np.cos(b) + np.cos(a)

t = 0.5*( (b-a)*Roots + a + b )
Integral = 0.5*(b-a)*np.sum(Weights*f(t))

#print(Integral)
#print(exact)

In [7]:
#según complementaria

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetLegendre(n,x,y):
    
    y = (x**2 - 1)**n
    
    poly = sym.diff( y,x,n )/(2**n*np.math.factorial(n))
    
    return poly

def GetLegendreRecursive(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = x
    else:
        poly = ((2*n-1)*x*GetLegendreRecursive(n-1,x)-(n-1)*GetLegendreRecursive(n-2,x))/n
   
    return sym.expand(poly,x)

def GetDLegendre(n,x):
    Pn = GetLegendreRecursive(n,x)
    return sym.diff(Pn,x,1)

def GetAllRootsGLeg(n):

    xn = np.linspace(-1,1,100)
    
    Legendre = []
    DLegendre = []
    
    for i in range(n+1):
        Legendre.append(GetLegendreRecursive(i,x))
        DLegendre.append(GetDLegendre(i,x))
    
    poly = sym.lambdify([x],Legendre[n],'numpy')
    Dpoly = sym.lambdify([x],DLegendre[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

def GetWeightsGLeg(n):

    Roots = GetAllRootsGLeg(n)
    DLegendre = []
    
    for i in range(n+1):
        DLegendre.append(GetDLegendre(i,x))
    
    Dpoly = sym.lambdify([x],DLegendre[n],'numpy')
    Weights = 2/((1-Roots**2)*Dpoly(Roots)**2)
    
    return Weights
"""
n = 6
Roots, Weights = np.polynomial.legendre.leggauss(n)
raices = GetAllRootsGLeg(n)
pesos = GetWeightsGLeg(n)
print(Roots)
print(raices)
print(Weights)
print(pesos)"""

"""funcion = lambda x: x**4
I = 0
for i in range(5):
    I += pesos[i]*funcion(raices[i])
I"""

'funcion = lambda x: x**4\nI = 0\nfor i in range(5):\n    I += pesos[i]*funcion(raices[i])\nI'

Gauss-Laguerre

In [8]:
#según magistral

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)
n = 6

def GetLaguerre(n,x,y):

    y = sym.exp(-x)*x**n
    
    poly = sym.exp(x)*sym.diff(y,x,n)/( np.math.factorial(n) )
    
    return poly

Laguerre = []

for i in range(n+1):
    
    Poly = GetLaguerre(i,x,y)
    Laguerre.append(Poly)

Roots,Weights = np.polynomial.laguerre.laggauss(n)

f1 = lambda x: np.cos(x)
I = np.sum( f1(Roots)*Weights )

f2 = lambda x: np.exp(-x)*np.cos(x)
exact = integrate.quad(f2,0,np.inf)

#print(I)
#print(exact)

AttributeError: 'function' object has no attribute 'quad'

In [ ]:
#según complementaria

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetLaguerre(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = -x + 1
    else:
        poly = ((2*n -1 -x)*GetLaguerre(n-1,x)-(n-1)*GetLaguerre(n-2,x))/n
   
    return sym.expand(poly,x)

def GetDLaguerre(n,x):
    Pn = GetLaguerre(n,x)
    return sym.diff(Pn,x,1)

def GetAllRootsGLag(n):
    
    xn = np.linspace(0,n+((n-1)*n**(1/2)),100)
    
    Laguerre = []
    DLaguerre = []
    
    for i in range(n+1):
        Laguerre.append(GetLaguerre(i,x))
        DLaguerre.append(GetDLaguerre(i,x))
    
    poly = sym.lambdify([x],Laguerre[n],'numpy')
    Dpoly = sym.lambdify([x],DLaguerre[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

def GetWeightsGLag(n):

    Roots = GetAllRootsGLag(n)

    Laguerre = []
    
    for i in range(n+2):
        Laguerre.append(GetLaguerre(i,x))
    
    poly = sym.lambdify([x],Laguerre[n+1],'numpy')
    Weights = Roots/(((n+1)**2)*(poly(Roots)**2))
    
    return Weights
"""
n = 6
Roots, Weights = np.polynomial.laguerre.laggauss(n)
raices = GetAllRootsGLag(n)
pesos = GetWeightsGLag(n)
print(Roots)
print(raices)
print(Weights)
print(pesos)"""

Gauss-Hermite

In [ ]:
#según magistral

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)
n = 6

def GetHermite(n,x,y):
      
    y = sym.exp(-x**2)
    
    poly = (-1)**n*sym.exp(x**2)*sym.diff(y,x,n)
    
    return poly

Hermite = []

for i in range(n+1):
    
    Poly = GetHermite(i,x,y)
    Hermite.append(Poly)

Roots, Weights = np.polynomial.hermite.hermgauss(n)

f = lambda x: 1/np.sqrt(np.pi) * 2 * x**4
I = np.sum(f(Roots)*Weights)

f2 = lambda x: 1/np.sqrt(np.pi) * 2 * x**4 * np.exp(-x**2)
exact = integrate.quad(f2,-np.inf,np.inf)

#print(I)
#print(exact)

In [ ]:
#según complementaria

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetHermite(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = (2*x)*GetHermite(n-1,x)-(2*(n-1))*GetHermite(n-2,x)

    return sym.expand(poly,x)

def GetDHermite(n,x):
    Pn = GetHermite(n,x)
    return sym.diff(Pn,x,1)

def GetAllRootsGHer(n):
    
    xn = np.linspace((-1)*((4*n)**(1/2)),(4*n)**(1/2),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)

    Hermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n-1],'numpy')
    Weights = ((2**(n-1))*(np.math.factorial(n))*(np.pi**(1/2)))/((n**2)*((poly(Roots)**2)))
    
    return Weights
"""
n = 6
Roots, Weights = np.polynomial.hermite.hermgauss(n)
raices = GetAllRootsGHer(n)
pesos = GetWeightsGHer(n)
print(Roots)
print(raices)
print(Weights)
print(pesos)"""

Integración de a-b usando Gauss-Legengre

In [ ]:
#También en Gauss-Legengre según la magistral hay un ejemplo de integración de una variable
def GetIntegral(a,b,f):
    n = 6
    sum = 0
    raices = GetAllRootsGLeg(n)
    pesos = GetWeightsGLeg(n)
    for k in range(n):
        sum += pesos[k] * f(((raices[k]*(b-a))/2) + ((b+a)/2))
        I = sum*((b-a)/2)
    return I
"""
a = 0
l = 5
funcion = lambda x: -x + l
exact = 0.5*(l**2)

print(GetIntegral(a,l,funcion))
print(exact)"""

def GetDoubleIntegral(a,b,c,d,f):
    n = 6
    sum = 0
    raices = GetAllRootsGLeg(n)
    pesos = GetWeightsGLeg(n)
    for j in range(n):
        for i in range(n):
          sum += (pesos[i]*pesos[j]) * f(((raices[i]*(b-a))/2) + ((b+a)/2) , ((raices[j]*(d-c))/2) + ((d+c)/2))
    I = sum*((b-a)/2)*((d-c)/2)
    return I
"""
def functionDouble(x,y):
    return x+(2*(y**2))

a = 1
b = 3
c = 1
d = 4
exact = 96

print(GetDoubleIntegral(a,b,c,d,functionDouble))
print(exact)"""

Método Montecarlo

$$
\begin{align*}
\int_a^b f(x) dx &\approx \frac{b-a}{n} \sum_{i=0}^{n-1} f(x_i)
\end{align*}
$$

In [ ]:
#1D

N = 1000000
a = 0
b = np.pi
x = np.random.uniform(a,b,N) #N es el número de muestras aleatorias que tomará

def func_integrate(x):
    return np.exp(-x)*np.sin(x)

fi = func_integrate(x)
I = (b-a)*sum(fi)/N

Iteo = 0.5*(1+np.exp(-np.pi))
#print(f'El valor teórico es: {Iteo:.6f} y valor utilizando Monte Carlo es {I:.6f}, con un error porcentual de: {np.abs(1-I/Iteo): .6%}')

In [ ]:
#Error en función de N

N = 10
Error = []
n = []
Integrals = []

for i in range(0,6):
    n.append(N)
    x = np.random.uniform(a,b,N)
    I = (b-a)*sum(func_integrate(x))/n[i]
    Integrals.append(I)
    error = round(np.abs(I-Iteo)/Iteo,3)*100
    Error.append(error)
    N *= 10
n = np.array(n)
Error = np.array(Error)

fig, ax = plt.subplots()
ax.plot(np.log10(n),Error,label='Error(%)')
ax.plot(np.log10(n),1/((n)**(1/2)),label='Cota superior')
ax.set_ylabel('Error(%)')
ax.set_xlabel('log10(N)')
plt.legend()

In [ ]:
#Integrales de orden superior

#2D

#Coordenadas cartesianas
R = 7
N = 10000
x = np.random.uniform(-R,R,N)
y = np.random.uniform(-R,R,N)

suma = 0

for i in range(N):
    if x[i]**2+y[i]**2<R**2:
        suma += 1 #Se suma uno puesto que la función es 1

I_coor_cartesian = (2*R)**2*suma/N

#Coordenadas polares
r = np.random.uniform(0,R,N)
theta = np.random.uniform(0,2*np.pi,N)

area_func = lambda x: x
f = area_func(r)

I_coor_polar = sum(f)*(R*2)*np.pi/N

Iteo = np.pi*(R**2)

#3D

#Region de integracion es la esfera de radio 1
R = 1
N = 10000
x = np.random.uniform(-R,R,N)
y = np.random.uniform(-R,R,N)
z = np.random.uniform(-R,R,N)

def function(x,y,z):
    f = np.sin(x**2 + y**2 + z**2)*(np.exp(x**2 + y**2 + z**2))
    return f

suma = 0
for i in range(N):
    if x[i]**2+y[i]**2+z[i]**2 < R**2:
        suma += function(x[i],y[i],z[i])

I = suma*((2*R)**3)/N

Derivada direccional

Álgebra Lineal

In [ ]:
#Multiplicación de matrices

def mulM(A,B):

    if len(np.shape(A)) == 1:
        A = np.reshape(A,(1,len(A)))

    if len(np.shape(B)) == 1:
        B = np.reshape(B,(len(B),1))

    AB = np.zeros(shape=(np.shape(A)[0],np.shape(B)[1]))

    if np.shape(A)[1] == np.shape(B)[0]:
        for i in range(np.shape(B)[1]):
            for j in range(np.shape(A)[0]):
                sum = 0
                for k in range(np.shape(A)[1]):
                    sum += A[j][k]*B[k][i]
                    
                AB[j][i] = sum
    else:
        AB = None
        print('Multiplicación no está definida')
    
    return AB

In [ ]:
#Eliminación Gaussiana y sustitucion

def gaussian_elimination(A: np.array,b: np.array):
    '''
    Args:
        A (np.array): Matriz de coeficientes cuadrada (n x n).
        b (np.array): Vector de constantes de longitud n

    Returns:
        M (np.array): Matriz aumentada [A|b] reducida utilizando el algoritmo de eliminación Gaussiana
    '''
    n = np.shape(A)[0]
    M = np.zeros(shape=(n,n+1))
    M[:,0:n] = A
    M[:,n] = b
    
    i_ = 1
    for j in range(0,n-1):
        
        for i in range(i_,n):
            #print(i,j)
            M[i,:] -= (M[i,j]/M[j,j])*M[j,:]
        i_ +=1

    return M

def back_substitution(A, b):
    '''
    Args:
        A (np.array): Matriz de coeficientes triangular superior
        b (np.arra): vector de constantes
        
    '''

    n = np.shape(A)[0]

    x = np.zeros(n)

    for i in range(n-1,-1,-1):
        sum = b[i]
        for j in range(n-1,i,-1):
            sum -= A[i,j]*x[j]
        x[i] = sum/A[i,i]

    return x

In [ ]:
#Valores y Vectores propios

#numpy

def GetValorAndVectorMax(A,v,its=11):
    
    for i in range(its):
        vn = v/(np.linalg.norm(v))
        v = mulM(A,vn)
        lam = mulM(vn.T,v)

    return lam,v/(np.linalg.norm(v))

def GetValorAndVectorMinor(A,v,its=11):
    A = np.linalg.inv(A)
    for i in range(its):
        vn = v/(np.linalg.norm(v))
        v = mulM(A,vn)
        lam = mulM(vn.T,v)

    return 1/lam,v/(np.linalg.norm(v))

#sympy

def GetValorAndVectorMax(A,v,its=11):
    v = Matrix(v)
    for i in range(its):
        vn = v.normalized()
        v = A*vn
        lam = vn.T*v
    
    v = v.normalized()

    return lam,v

def GetValorAndVectorMinor(A,v,its=11):
    A = A.inv()
    v = Matrix(v)
    for i in range(its):
        vn = v.normalized()
        v = A*vn
        lam = vn.T*v

    v = v.normalized()

    return 1/lam, v